## Hyperparam Tuning

Now that we know which models are performing better, it's time to perform cross validation and tune hyperparameters.
- Do a google search for hyperparameter ranges for each type of model.

GridSearch/RandomSearch are a great methods for checking off both of these tasks.

There is a fairly significant issue with this approach for this particular problem (described below). But in the interest of creating a basic functional pipeline, you can just use the default Sklearn methods for now.

## Preventing Data Leakage in Tuning - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it, if you have time!**

BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood. This is a challenging, real world data problem! To help you out, we've created some psuedocode and docstrings to get you started. 

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

This psuedocode may help you fill in the function:

```python
kfold = KFold() # fit sklearn k folds on X_train
train_folds = []
val_folds = []
for training_index, val_index in kfold.split(X_train):
    train_fold, val_fold = #.iloc loop variables on X_train

    # recompute training city means like you did in notebook 1 
    # merge to validation fold
        
    train_folds.append(train_fold)
    val_folds.append(val_fold)

    return train_folds, val_folds
```


**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.


This psuedocode may help you fill in the function:

```python
hyperparams = # Generate hyperparam options with itertools
hyperparam-scores = []
for hyperparam-combo in hyperparams:

    scores = []

    for folds in allmyfolds:
        # score fold the fold with the model/ hyperparams
        scores.append(score-fold)
        
    score = scores.mean()
    hyperparam-scores.append(score)
# After loop, find max of hyperparam-scores. Best params are at same index in `hyperparams` loop iteratble
```

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [1]:
# import models and fit
import pandas as pd
import numpy as np
import itertools
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import KFold
import joblib

In [2]:
# develop your custom functions here

def custom_cross_validation(training_data, target_column, n_splits =5):
    '''creates n_splits sets of training and validation folds

    Args:
      training_data: the dataframe of features and target to be divided into folds
      n_splits: the number of sets of folds to be created

    Returns:
      A tuple of lists, where the first index is a list of the training folds, 
      and the second the corresponding validation fold

    Example:
        >>> output = custom_cross_validation(train_df, n_splits = 10)
        >>> output[0][0] # The first training fold
        >>> output[1][0] # The first validation fold
        >>> output[0][1] # The second training fold
        >>> output[1][1] # The second validation fold... etc.
    '''

    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    training_folds = []
    validation_folds = []

    for train_index, val_index in kfold.split(training_data):
        train_fold = training_data.iloc[train_index].copy()
        val_fold = training_data.iloc[val_index].copy()

        training_folds.append(train_fold)
        validation_folds.append(val_fold)
        
    return training_folds, validation_folds


    return training_folds, validation_folds

def hyperparameter_search(training_folds, validation_folds, model, target_column, param_grid):
    '''outputs the best combination of hyperparameter settings in the param grid, 
    given the training and validation folds

    Args:
      training_folds: the list of training fold dataframes
      validation_folds: the list of validation fold dataframes
      param_grid: the dictionary of possible hyperparameter values for the chosen model

    Returns:
      A list of the best hyperparameter settings based on the chosen metric

    Example:
        >>> param_grid = {
          'max_depth': [None, 10, 20, 30],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 4],
          'max_features': ['sqrt', 'log2']} # for random forest
        >>> hyperparameter_search(output[0], output[1], param_grid = param_grid) 
        # assuming 'ouput' is the output of custom_cross_validation()
        [20, 5, 2, 'log2'] # hyperparams in order
    '''

    hyperparams = list(itertools.product(*param_grid.values()))
    best_score = float('inf')
    best_params = None

    for params in hyperparams:
        param_dict = dict(zip(param_grid.keys(), params))
        model.set_params(**param_dict)
        scores = []

        for train_fold, val_fold in zip(train_folds, val_folds):
            X_train_fold = train_fold.drop(columns=[target_column])
            y_train_fold = train_fold[target_column]
            X_val_fold = val_fold.drop(columns=[target_column])
            y_val_fold = val_fold[target_column]

            model.fit(X_train_fold, y_train_fold)
            predictions = model.predict(X_val_fold)
            score = np.sqrt(mean_squared_error(y_val_fold, predictions))  # RMSE
            scores.append(score)

        avg_score = sum(scores) / len(scores)

        if avg_score < best_score:
            best_score = avg_score
            best_params = param_dict

    return best_params
    


## Hyperparam Tuning

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from functions_variables import custom_cross_validation, hyperparameter_search

# Load the training dataset
dataframe_train = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/DS-Midterm-Project/processed/train.csv')
X_train_df = dataframe_train.drop(columns=['description.sold_price'])
y_train = dataframe_train['description.sold_price']
print("Training set shape:", X_train_df.shape, y_train.shape)

# Load the test dataset
dataframe_test = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/DS-Midterm-Project/processed/test.csv')
X_test_df = dataframe_test.drop(columns=['description.sold_price'])
y_test = dataframe_test['description.sold_price']
print("Test set shape:", X_test_df.shape, y_test.shape)

# Define target column
target_column = 'description.sold_price'

# Perform custom cross-validation
train_folds, val_folds = custom_cross_validation(dataframe_train, target_column, n_splits=5)

# Define the model and parameter grid
model = GradientBoostingRegressor()

# Define the parameter grid
param_grid = {
    'n_estimators': [300],
    'max_depth': [3],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Perform hyperparameter search
best_params = hyperparameter_search(train_folds, val_folds, model, param_grid, target_column)
print(f"Best Parameters: {best_params}")


# Train the best model on the entire training data
best_model = GradientBoostingRegressor(**best_params)
X_train = dataframe_train.drop(columns=[target_column])
y_train = dataframe_train[target_column]
best_model.fit(X_train, y_train)

# Load the test data
dataframe_test = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/DS-Midterm-Project/processed/test.csv')
X_test = dataframe_test.drop(columns=[target_column])
y_test = dataframe_test[target_column]

# Make predictions
predictions = best_model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
adjusted_r2 = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - X_test_df.shape[1] - 1)

# Print metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R²: {r2}")
print(f"Adjusted R²: {adjusted_r2}")

Training set shape: (902, 14) (902,)
Test set shape: (226, 14) (226,)
Best Parameters: {'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.1, 'subsample': 0.8, 'min_samples_split': 2, 'min_samples_leaf': 2}
Mean Squared Error (MSE): 6559054951.352636
Root Mean Squared Error (RMSE): 80987.99263688807
Mean Absolute Error (MAE): 56845.11360414063
R²: 0.8115102693304868
Adjusted R²: 0.7990038417031257


We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

In [6]:
# save your best model here
joblib.dump(best_model, 'best_model.pkl')

['best_model.pkl']

In [5]:
with open('models/model.pkl','wb') as f:
    pickle.dump(model,f)

FileNotFoundError: [Errno 2] No such file or directory: 'models/model.pkl'

## Building a Pipeline (Stretch)

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it if you have time!**

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [ ]:
# Build pipeline here
#Import custom functions
# from functions_variables import custom_cross_validation, hyperparameter_search

# Load your data
dataframe_train_poly = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/DS-Midterm-Project/processed/train.csv')
target_column = 'description.sold_price'
city_column = 'city_encoded'


# Perform custom cross-validation
#train_folds, val_folds = custom_cross_validation(dataframe_train_poly, target_column, city_column, n_splits=5)

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize the features
    ('model', GradientBoostingRegressor())  # XGBoost model
])

# Define the parameter grid
param_grid = {
    'model__n_estimators': [300],
    'model__max_depth': [3],
    'model__learning_rate': [0.1],
    'model__subsample': [0.8],
    'model__min_samples_split': [5],
    'model__min_samples_leaf': [1]
}

# Perform hyperparameter search
best_params = hyperparameter_search(train_folds, val_folds, pipeline, param_grid, target_column)
print(f"Best Parameters: {best_params}")

# Update the pipeline with the best parameters
pipeline.set_params(**best_params)

# Train the best pipeline on the entire training data
X_train = dataframe_train_poly.drop(columns=[target_column])
y_train = dataframe_train_poly[target_column]
pipeline.fit(X_train, y_train)

# Save the pipeline
joblib.dump(pipeline, 'best_pipeline.pkl')

# Load the test data
dataframe_test_poly = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/DS-Midterm-Project/processed/test.csv')
X_test = dataframe_test_poly.drop(columns=[target_column])
y_test = dataframe_test_poly[target_column]

# Make predictions
predictions = pipeline.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
adjusted_r2 = 1 - (1 - r2) * (len(y_test) - 1) / (len(y_test) - X_test_df.shape[1] - 1)

# Print metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R²: {r2}")
print(f"Adjusted R²: {adjusted_r2}")


Best Parameters: {'model__n_estimators': 300, 'model__max_depth': 3, 'model__learning_rate': 0.1, 'model__subsample': 0.8, 'model__min_samples_split': 5, 'model__min_samples_leaf': 1}
Mean Squared Error (MSE): 6203684591.871016
Root Mean Squared Error (RMSE): 78763.47244675679
Mean Absolute Error (MAE): 54772.19177984647
R²: 0.8217226648422525
Adjusted R²: 0.8098938369170939


Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [ ]:
# save your pipeline here
joblib.dump(pipeline, 'best_pipeline.pkl')

['best_pipeline.pkl']